## Deliverable 2. Create a Customer Travel Destinations Map.

In [66]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [28]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Shakawe,BW,-18.3667,21.8500,72.61,29,80,4.18,broken clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,40.33,72,100,1.66,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,40.66,45,0,14.97,clear sky
3,3,Barrow,US,71.2906,-156.7887,39.22,87,0,11.50,clear sky
4,4,Sur,OM,22.5667,59.5289,93.61,57,91,7.87,overcast clouds


In [29]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [31]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
new_city_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Praya,CV,14.9215,-23.5087,79.34,61,75,17.27,broken clouds
7,7,Butaritari,KI,3.0707,172.7902,82.72,76,76,20.45,light rain
9,9,Sola,VU,-13.8833,167.5500,80.87,77,11,24.09,few clouds
10,10,East London,ZA,-33.0153,27.9116,78.22,31,0,10.31,clear sky
11,11,Kismayo,SO,-0.3582,42.5454,77.54,72,54,18.72,broken clouds
...,...,...,...,...,...,...,...,...,...,...
667,667,Sokoni,TZ,-6.4333,39.5500,75.63,76,14,13.29,light rain
668,668,Mitsamiouli,KM,-11.3847,43.2844,77.07,70,43,7.87,scattered clouds
671,671,Odweyne,SO,9.4092,45.0640,86.81,31,32,21.00,scattered clouds
672,672,Caravelas,BR,-17.7125,-39.2481,79.21,59,66,8.05,broken clouds


In [33]:
# 4a. Determine if there are any empty rows.
new_city_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = new_city_df.dropna()
clean_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [35]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Praya,CV,79.34,broken clouds,14.9215,-23.5087,
7,Butaritari,KI,82.72,light rain,3.0707,172.7902,
9,Sola,VU,80.87,few clouds,-13.8833,167.5500,
10,East London,ZA,78.22,clear sky,-33.0153,27.9116,
11,Kismayo,SO,77.54,broken clouds,-0.3582,42.5454,
16,Samokov,BG,77.50,scattered clouds,42.3333,23.5500,
20,Albany,US,76.68,overcast clouds,42.6001,-73.9662,
23,Robertsport,LR,78.53,light rain,6.7533,-11.3686,
24,Bambanglipuro,ID,77.23,scattered clouds,-7.9500,110.2833,
29,Bud,US,81.68,scattered clouds,39.4470,-86.1758,


In [62]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        pass

In [70]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna()
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291 entries, 5 to 674
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 291 non-null    object 
 1   Country              291 non-null    object 
 2   Max Temp             291 non-null    float64
 3   Current Description  291 non-null    object 
 4   Lat                  291 non-null    float64
 5   Lng                  291 non-null    float64
 6   Hotel Name           291 non-null    object 
dtypes: float64(3), object(4)
memory usage: 26.3+ KB


In [72]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [88]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [89]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))